# MCDC solver

In [2]:
class Condition:
    def __init__(self):
        self.name = ''
        self.precop = ''
        self.trailop = ''

In [3]:
class Testcase:
    def __init__(self):
        self.seq = ''
        self.res = ''

In [4]:
class Decision:
    def __init__(self):
        self.conditions = list()
        self.testcases = list()
        
    def extract(self, raw):
        elem = list(raw)
        size = len(raw)
        i = 0
        while i < size:
            temp = Condition()
            temp.name = elem[i]
            if i == 0:
                temp.precop = '-'
            else:
                temp.precop = elem[i-1]
            if (i+1) == size:
                temp.trailop = '-'
            else:
                temp.trailop = elem[i+1]
            self.conditions.append(temp)
            i += 2
    
    def derivation(self):
        trueTestcases = list()
        falseTestcases = list()
        testcase = list()
        length = len(self.conditions)
        for i in range(0, len(self.conditions)):
            for x in range(0, length - (length - i)):
                if self.conditions[x].trailop == "&":
                    testcase.append("1")
                else:
                    testcase.append("0")
            testcase.append("1")
            for x in range(i+1, length):
                if self.conditions[x].precop == "&":
                    testcase.append("1")
                else:
                    testcase.append("0")
            trueTestcases.append(Testcase())
            trueTestcases[i].seq = ''.join(testcase)
            trueTestcases[i].res = "True"
            testcase[i]=("0")
            falseTestcases.append(Testcase())
            falseTestcases[i].seq = ''.join(testcase)
            falseTestcases[i].res= "False"
            testcase.clear()
        self.testcases = trueTestcases + falseTestcases
    
    def reduce(self):
        li = list()
        temp = list()
        for testcase in self.testcases:
            if testcase.seq not in li:
                li.append(testcase.seq)
                temp.append(testcase)
        self.testcases.clear()
        self.testcases = temp
        
    def print_testcases(self):
        i = 1
        for x in self.testcases:
            print('Testcase ' + str(i) + ': ' + x.seq + '   ' + x.res)
            i += 1

In [5]:
s1 = 'A&B|C&D|C|E|F&G&H&I|J'

decision = Decision()
decision.extract(s1)
decision.derivation()
decision.reduce()
decision.print_testcases()

Testcase 1: 11010001110   True
Testcase 2: 10110001110   True
Testcase 3: 10101001110   True
Testcase 4: 10100101110   True
Testcase 5: 10100011110   True
Testcase 6: 10100011101   True
Testcase 7: 01010001110   False
Testcase 8: 10010001110   False
Testcase 9: 10100001110   False
Testcase 10: 10100010110   False
Testcase 11: 10100011010   False
Testcase 12: 10100011100   False
